In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
pd.set_option("display.float_format", "{:.12g}".format)
np.set_printoptions(suppress=True, precision=12) 

In [2]:
def readfile(input_filename):

     
     # Open and parse the file
     with open(input_filename, "r") as file:
          for line in file:
              
               if line.startswith("=== EVENT 1 ==="):
                    nextline = next(file)
                    #print(nextline)
                    nextline.strip()
                    start = '== TDC corrected time =' 
                    end = '=='
                    run_starttime = ((nextline.split(start))[1].split(end)[0])
                    #print(run_starttime)

               if line.startswith("=== EVENT 10000 ==="):
                    nextline = next(file)
                    #print(nextline)
                    nextline.strip()
                    start = '== TDC corrected time =' 
                    end = '=='
                    run_endtime = ((nextline.split(start))[1].split(end)[0])
                    #print(run_endtime)

     return run_starttime, run_endtime

               

In [3]:
def parse_time(time_str):
    """Extract hours, minutes, seconds, and nanoseconds from a given time string."""
    match = re.match(r"(\d+)h(\d+)m(\d+)s,(\d+)\.(\d+)\.(\d+)ns", time_str)
    if match:
        hours, minutes, seconds, ns1, ns2, ns3 = map(int, match.groups())
        nanoseconds = (hours * 3600 + minutes * 60 + seconds) * 1_000_000_000 + ns1 * 1_000_000 + ns2 * 1_000 + ns3
        return nanoseconds
    else:
        raise ValueError("Invalid time format")


In [4]:
def format_time(nanoseconds):
    """Convert nanoseconds to a human-readable h:m:s,ns format."""
    total_seconds, ns = divmod(nanoseconds, 1_000_000_000)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours}h{minutes}m{seconds}s,{ns:09d}ns"

In [5]:
#define path to files
file_path="/Users/bordonis/ResearchActivities/Perovskite/data/NewSiPM_KEPboard_DPNCboard/CERN/Cs104-A/"

#list of filenames
filelist = ["Run_006_CERN_Cs104-A_Data_10_14_2024_Ascii.dat", 
            "Run_010_CERN_Cs104-A_Data_10_15_2024_Ascii.dat", 
            "Run_015_CERN_Cs104-A_Data_10_15_2024_Ascii.dat"]


In [6]:
#define output as a dictionary 

output = {}

for f in filelist:

    starttime, endtime = readfile(file_path+f)
    
    # Convert to nanoseconds
    start_ns = parse_time(starttime.strip())
    end_ns = parse_time(endtime.strip())

    # Compute difference
    diff_ns = end_ns - start_ns

    # Format the result
    time_diff_str = format_time(diff_ns)


    output[f] = time_diff_str

    

In [7]:
# format output 
pd.set_option('max_colwidth', 100)
pd.DataFrame(output.items(), columns=['File', 'Time duration'])
print(pd.DataFrame(output.items(), columns=['File', 'Time duration']))


                                             File         Time duration
0  Run_006_CERN_Cs104-A_Data_10_14_2024_Ascii.dat  1h20m12s,700686730ns
1  Run_010_CERN_Cs104-A_Data_10_15_2024_Ascii.dat  0h32m23s,655881838ns
2  Run_015_CERN_Cs104-A_Data_10_15_2024_Ascii.dat   0h15m4s,798348820ns
